In [111]:
import pandas as pd
import numpy as np
import geopandas as gp
from geopy.geocoders import Nominatim

In [112]:
taxi_zone_df = pd.read_csv('https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv')

taxi_zone_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [113]:
taxi_zone_df.tail()

,LocationID,Borough,Zone,service_zone
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN
264,265,NaN,Outside of NYC,NaN


In [114]:
taxi_zone_df['service_zone'].value_counts()

service_zone
Boro Zone      205
Yellow Zone     55
Airports         2
EWR              1
Name: count, dtype: int64

In [115]:
zone_data = pd.read_parquet('https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet')

In [116]:
zone_data.tail()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
258,259,0.12675030519,0.00039455249,Woodlawn/Wakefield,259,Bronx,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe6\x00...
259,260,0.13351415464,0.00042234533,Woodside,260,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x10\x01...
260,261,0.02712045636,0.00003434232,World Trade Center,261,Manhattan,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00F\x00\x0..."
261,262,0.04906362315,0.00012233027,Yorkville East,262,Manhattan,"b""\x01\x06\x00\x00\x00\x02\x00\x00\x00\x01\x03..."
262,263,0.03701662530,0.00006576977,Yorkville West,263,Manhattan,b'\x01\x03\x00\x00\x00\x01\x00\x00\x000\x00\x0...


In [117]:
len(zone_data)

263

In [118]:
zone_data.head(20)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.11635745319,0.00078230679,Newark Airport,1,EWR,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe8\x00...
1,2,0.43346966679,0.00486634038,Jamaica Bay,2,Queens,b'\x01\x06\x00\x00\x00!\x00\x00\x00\x01\x03\x0...
2,3,0.08434110590,0.00031441416,Allerton/Pelham Gardens,3,Bronx,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00y\x00\x0...
3,4,0.04356652709,0.00011187195,Alphabet City,4,Manhattan,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00X\x00\x0...
4,5,0.09214648986,0.00049795749,Arden Heights,5,Staten Island,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xaa\x00...
5,6,0.15049054252,0.00060646098,Arrochar/Fort Wadsworth,6,Staten Island,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x15\x01...
6,7,0.10741717112,0.00038978799,Astoria,7,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xb6\x00...
7,8,0.02759069116,0.00002658772,Astoria Park,8,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00(\x00\x0...
8,9,0.09978409247,0.00033844380,Auburndale,9,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xbd\x00...
9,10,0.09983947942,0.00043582382,Baisley Park,10,Queens,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x9d\x00...


In [119]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')
mar_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')
apr_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet')
may_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet')
jun_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet')
jul_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet')
aug_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet')
sep_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet')
oct_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet')
nov_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-11.parquet')
dec_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-12.parquet')


In [120]:
jan_df.head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.50,0.5,6.90,0.00,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.00,0.5,10.00,0.00,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.50,0.5,0.00,0.00,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.00,0.5,0.00,0.00,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.00,0.5,0.00,0.00,1.0,8.00,2.5,0.00


In [121]:
dfs = [jan_df, feb_df, mar_df, apr_df, may_df, jun_df, jul_df, aug_df, sep_df, oct_df, nov_df, dec_df]

ny_taxi_2024_df = pd.concat(dfs)

In [122]:
print(f'{len(ny_taxi_2024_df):,.2f}')

41,169,720.00


In [123]:
display(ny_taxi_2024_df.head(10))
display(ny_taxi_2024_df.tail(10))

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.5,0.5,6.90,0.0,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.0,0.5,10.00,0.0,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.0,0.5,0.00,0.0,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.0,0.5,0.00,0.0,1.0,8.00,2.5,0.00


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,NaN,1.60,NaN,None,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,NaN
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,NaN,2.47,NaN,None,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,NaN
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,NaN,1.96,NaN,None,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,NaN
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,NaN,2.90,NaN,None,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,NaN
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,NaN,2.49,NaN,None,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,NaN
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,NaN,10.71,NaN,None,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,NaN,4.56,NaN,None,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,NaN,3.94,NaN,None,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,NaN,4.20,NaN,None,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,NaN,5.76,NaN,None,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,NaN


In [124]:
len(ny_taxi_2024_df[ny_taxi_2024_df["PULocationID"] == 264].value_counts())

119824

In [125]:
len(ny_taxi_2024_df[ny_taxi_2024_df["DOLocationID"] == 264].value_counts())

182294

In [126]:
len(ny_taxi_2024_df[(ny_taxi_2024_df["PULocationID"] == 264) & (ny_taxi_2024_df["DOLocationID"] != 264)].value_counts())

49799

In [127]:
len(ny_taxi_2024_df[(ny_taxi_2024_df["DOLocationID"] == 264) & (ny_taxi_2024_df["PULocationID"] != 264)].value_counts())

112269

In [128]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0     34651027
2.0      1406870
99.0      466974
5.0       321948
3.0       129951
4.0       101642
6.0           76
Name: count, dtype: int64

In [129]:
ny_taxi_2024_df['passenger_count'].isna().sum()/len(ny_taxi_2024_df['passenger_count'])

0.09937478321445956

In [130]:
ny_taxi_2024_df['RatecodeID'].isna().sum()/len(ny_taxi_2024_df['RatecodeID'])

0.09937478321445956

In [131]:
ny_taxi_2024_df['congestion_surcharge'].isna().sum()/len(ny_taxi_2024_df['congestion_surcharge'])

0.09937478321445956

In [132]:
ny_taxi_2024_df['Airport_fee'].isna().sum()/len(ny_taxi_2024_df['Airport_fee'])

0.09937478321445956

In [133]:
ny_taxi_2024_df['store_and_fwd_flag'].isna().sum()/len(ny_taxi_2024_df['store_and_fwd_flag'])

0.09937478321445956

In [134]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.09937478321445956
0.0
0.09937478321445956
0.09937478321445956
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.09937478321445956


In [135]:
ny_taxi_2024_df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
dtype: object

In [136]:
ny_taxi_2024_df['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
N    36903316
Y      175172
Name: count, dtype: int64

In [137]:
ny_taxi_2024_df['store_and_fwd_flag'] = ny_taxi_2024_df['store_and_fwd_flag'].fillna('N')

In [138]:
ny_taxi_2024_df['passenger_count'].value_counts()

passenger_count
1.0    28632703
2.0     5410774
3.0     1282073
4.0      814889
0.0      401354
5.0      320613
6.0      215798
8.0         192
7.0          56
9.0          36
Name: count, dtype: int64

In [139]:
def impute_passenger_count(ny_taxi_2024_df):
    ny_taxi_2024_df['passenger_count'] = ny_taxi_2024_df['passenger_count'].fillna(1)
    pass_mask = (ny_taxi_2024_df['passenger_count'] == 0)

    ny_taxi_2024_df.loc[pass_mask, 'passenger_count'] = 1

    return ny_taxi_2024_df


In [140]:
impute_passenger_count(ny_taxi_2024_df)

ny_taxi_2024_df['passenger_count'].value_counts()

passenger_count
1.0    33125289
2.0     5410774
3.0     1282073
4.0      814889
5.0      320613
6.0      215798
8.0         192
7.0          56
9.0          36
Name: count, dtype: int64

In [141]:
#ny_taxi_2024_df['passenger_count'] = ny_taxi_2024_df['passenger_count'].fillna(1)

In [142]:
ny_taxi_2024_df.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,1.0,1.60,NaN,N,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,NaN
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,1.0,2.47,NaN,N,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,NaN
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,1.0,1.96,NaN,N,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,NaN
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,1.0,2.90,NaN,N,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,NaN
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,1.0,2.49,NaN,N,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,NaN
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,NaN


In [143]:
start_date = pd.to_datetime('2024-12-01 00:00:00')
end_date = pd.to_datetime('2024-12-31 23:33:34')

In [144]:
december_df = ny_taxi_2024_df.loc[(ny_taxi_2024_df['tpep_pickup_datetime'] >= start_date) & (ny_taxi_2024_df['tpep_pickup_datetime'] <= end_date)]
display(december_df.head())
display(december_df.tail())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3178038,2,2024-12-01 00:04:01,2024-12-01 00:15:11,1.0,1.97,1.0,N,114,170,1,12.8,1.0,0.5,3.56,0.0,1.0,21.36,2.5,0.0
3220341,2,2024-12-01 15:35:18,2024-12-01 15:56:58,1.0,2.74,1.0,N,237,142,1,20.5,0.0,0.5,4.90,0.0,1.0,29.40,2.5,0.0
3220342,2,2024-12-01 15:57:57,2024-12-01 16:12:50,1.0,2.61,1.0,N,142,263,1,16.3,0.0,0.5,4.06,0.0,1.0,24.36,2.5,0.0
3227441,2,2024-12-01 16:34:09,2024-12-01 16:41:29,1.0,1.59,1.0,N,143,48,1,10.0,0.0,0.5,0.00,0.0,1.0,14.00,2.5,0.0
3227442,2,2024-12-01 16:57:49,2024-12-01 17:15:24,1.0,2.26,1.0,N,68,224,1,17.0,0.0,0.5,5.25,0.0,1.0,26.25,2.5,0.0


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.0,1.0,-2.63,NaN,NaN
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.0,1.0,-0.18,NaN,NaN
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,NaN,NaN
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,NaN,NaN
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.0,1.0,6.17,NaN,NaN


In [145]:
for column in december_df:

    print(december_df[column].isna().sum()/len(december_df[column]))

0.0
0.0
0.0
0.0
0.0
0.08895266870822686
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08895266870822686
0.08895266870822686


In [146]:
def find_airport_zones_vectorized(taxi_zone_df):
    airport_mask = taxi_zone_df['Zone'].str.contains('Airport', case=False, na=False)
    airport_rows = taxi_zone_df[airport_mask].to_dict(orient='records')
    return airport_rows

In [147]:
find_airport_zones_vectorized(taxi_zone_df)

[{'LocationID': 1,
  'Borough': 'EWR',
  'Zone': 'Newark Airport',
  'service_zone': 'EWR'},
 {'LocationID': 132,
  'Borough': 'Queens',
  'Zone': 'JFK Airport',
  'service_zone': 'Airports'},
 {'LocationID': 138,
  'Borough': 'Queens',
  'Zone': 'LaGuardia Airport',
  'service_zone': 'Airports'}]

In [148]:
ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
 0.00    33765255
 1.75     3213953
-1.75       99253
 1.25          27
Name: count, dtype: int64

In [149]:
def impute_airport_fee(ny_taxi_2024_df):
    
    pu_mask = (ny_taxi_2024_df['PULocationID'] == 132) | (ny_taxi_2024_df['PULocationID'] == 138)
    fee_mask = np.isnan(ny_taxi_2024_df['Airport_fee']) | (ny_taxi_2024_df['Airport_fee'] == 1.25) | (ny_taxi_2024_df['Airport_fee'] == -1.75)
    mask = pu_mask & fee_mask
    ny_taxi_2024_df.loc[mask, 'Airport_fee'] = 1.75
    return ny_taxi_2024_df

In [150]:
impute_airport_fee(ny_taxi_2024_df)

ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
 0.00    33765255
 1.75     3323271
-1.75        2800
 1.25           4
Name: count, dtype: int64

In [151]:
def impute_outliers_airport_fee(ny_taxi_2024_df):
    
    pu_mask = (
        ny_taxi_2024_df['PULocationID'] != 132) | (
        ny_taxi_2024_df['PULocationID'] != 138) | (
            ny_taxi_2024_df['DOLocationID'] != 132) | (ny_taxi_2024_df['DOLocationID'] != 138)
    fee_mask = np.isnan(
        ny_taxi_2024_df['Airport_fee']) | (
        ny_taxi_2024_df['Airport_fee'] == 1.25) | (
            ny_taxi_2024_df['Airport_fee'] == -1.75)
    mask = pu_mask & fee_mask
    ny_taxi_2024_df.loc[mask, 'Airport_fee'] = 0
    return ny_taxi_2024_df

In [152]:
impute_outliers_airport_fee(ny_taxi_2024_df)

ny_taxi_2024_df['Airport_fee'].value_counts()

Airport_fee
0.00    37846449
1.75     3323271
Name: count, dtype: int64

In [153]:
ny_taxi_2024_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [154]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.0


In [155]:
ny_taxi_2024_df.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,1.0,1.60,NaN,N,188,61,0,13.28,0.0,0.5,0.00,0.00,1.0,14.78,NaN,0.0
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,1.0,2.47,NaN,N,37,37,0,-0.90,0.0,0.5,0.00,0.00,1.0,0.60,NaN,0.0
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,1.0,1.96,NaN,N,137,79,0,-7.72,0.0,0.5,0.00,0.00,1.0,-3.72,NaN,0.0
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,1.0,2.90,NaN,N,145,164,0,21.94,0.0,0.5,0.00,6.94,1.0,32.88,NaN,0.0
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,1.0,2.49,NaN,N,37,36,0,-0.38,0.0,0.5,0.00,0.00,1.0,6.12,NaN,0.0
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,NaN,N,16,7,0,-4.13,0.0,0.5,0.00,0.00,1.0,-2.63,NaN,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,NaN,N,252,16,0,-1.68,0.0,0.5,0.00,0.00,1.0,-0.18,NaN,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,NaN,N,181,255,0,4.46,0.0,0.5,5.19,0.00,1.0,11.15,NaN,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,NaN,N,165,61,0,27.07,0.0,0.5,0.00,0.00,1.0,28.57,NaN,0.0
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,1.0,5.76,NaN,N,225,114,0,2.17,0.0,0.5,0.00,0.00,1.0,6.17,NaN,0.0


In [156]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0     34651027
2.0      1406870
99.0      466974
5.0       321948
3.0       129951
4.0       101642
6.0           76
Name: count, dtype: int64

In [157]:
ny_taxi_2024_df[ny_taxi_2024_df['RatecodeID'] == 4].head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
177,2,2024-01-01 00:16:21,2024-01-01 00:46:36,1.0,23.00,4.0,N,70,265,2,125.5,6.00,0.5,0.00,0.00,1.0,134.75,0.0,1.75
1035,2,2024-01-01 00:47:05,2024-01-01 01:03:14,3.0,4.18,4.0,N,48,265,2,22.6,1.00,0.0,0.00,12.75,1.0,37.35,0.0,0.00
1316,2,2024-01-01 00:00:11,2024-01-01 00:42:33,1.0,9.89,4.0,N,234,265,2,81.4,1.00,0.0,0.00,12.75,1.0,96.15,0.0,0.00
2701,2,2024-01-01 00:48:30,2024-01-01 01:09:02,1.0,15.79,4.0,N,132,15,4,-91.9,-1.00,-0.5,0.00,0.00,-1.0,-96.15,0.0,1.75
2702,2,2024-01-01 00:48:30,2024-01-01 01:09:02,1.0,15.79,4.0,N,132,16,4,91.9,1.00,0.5,0.00,0.00,1.0,96.15,0.0,1.75
3233,2,2024-01-01 00:13:49,2024-01-01 00:42:27,2.0,25.13,4.0,N,132,265,1,108.7,1.00,0.5,0.00,6.94,1.0,119.89,0.0,1.75
3508,2,2024-01-01 00:24:30,2024-01-01 01:13:09,1.0,27.08,4.0,N,163,265,1,138.8,1.00,0.5,29.37,3.05,1.0,176.22,2.5,0.00
3600,2,2024-01-01 00:35:44,2024-01-01 01:19:49,2.0,36.75,4.0,N,138,265,1,220.7,6.00,0.5,45.64,0.00,1.0,275.59,0.0,1.75
3906,2,2024-01-01 00:47:53,2024-01-01 01:29:06,4.0,26.71,4.0,N,170,265,2,125.5,1.00,0.5,0.00,6.94,1.0,137.44,2.5,0.00
3929,1,2024-01-01 00:32:49,2024-01-01 00:54:05,1.0,10.00,4.0,N,132,265,1,45.7,2.75,0.5,12.50,0.00,1.0,62.45,0.0,1.75


In [158]:
def find_ratecode_zones_vectorized(taxi_zone_df):
    pattern = r'JFK|Newark|Nassau|Westchester'
    ratecode_rows = taxi_zone_df['Zone'].str.contains(pattern, case=False, na=False)
    ratecode_rows = taxi_zone_df[ratecode_rows].to_dict(orient='records')
    return ratecode_rows

In [159]:
taxi_zone_df[taxi_zone_df['Zone'].str.contains('Nassau', case=False, na=False)].head()

,LocationID,Borough,Zone,service_zone


In [160]:
find_ratecode_zones_vectorized(taxi_zone_df)

[{'LocationID': 1,
  'Borough': 'EWR',
  'Zone': 'Newark Airport',
  'service_zone': 'EWR'},
 {'LocationID': 132,
  'Borough': 'Queens',
  'Zone': 'JFK Airport',
  'service_zone': 'Airports'},
 {'LocationID': 250,
  'Borough': 'Bronx',
  'Zone': 'Westchester Village/Unionport',
  'service_zone': 'Boro Zone'}]

In [161]:
def impute_na_ratecodeID(ny_taxi_2024_df):
    def_mask = (ny_taxi_2024_df['PULocationID'] != ny_taxi_2024_df['PULocationID'].isin([np.isnan, 1, 132, 250, 265])) | (
        ny_taxi_2024_df['DOLocationID'] != ny_taxi_2024_df['DOLocationID'].isin([np.isnan, 1, 132, 250, 265])
    )
    rate_mask = (ny_taxi_2024_df['RatecodeID'] > 6) | np.isnan(ny_taxi_2024_df['RatecodeID'])
    mask = def_mask & rate_mask
    ny_taxi_2024_df.loc[mask, ['RatecodeID']] = 1
    return ny_taxi_2024_df

In [162]:
impute_na_ratecodeID(ny_taxi_2024_df)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,1.0,N,16,7,0,-4.13,0.0,0.5,0.00,0.0,1.0,-2.63,NaN,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,1.0,N,252,16,0,-1.68,0.0,0.5,0.00,0.0,1.0,-0.18,NaN,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,1.0,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,NaN,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,1.0,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,NaN,0.0


In [163]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0    39209213
2.0     1406870
5.0      321948
3.0      129951
4.0      101642
6.0          76
Name: count, dtype: int64

In [164]:
def impute_ratecodeIDs(ny_taxi_2024_df):
    mask = (ny_taxi_2024_df['PULocationID'] == 1) | (ny_taxi_2024_df['DOLocationID'] == 1)
    mask_2 = (ny_taxi_2024_df['PULocationID'] == 132) | (ny_taxi_2024_df['DOLocationID'] == 132)
    mask_3 = (ny_taxi_2024_df['PULocationID'] == 250) | (ny_taxi_2024_df['DOLocationID'] == 250) | (
        ny_taxi_2024_df['PULocationID'] == 265) | (ny_taxi_2024_df['DOLocationID'] == 265)
    ny_taxi_2024_df.loc[mask, ['RatecodeID']] = 3
    ny_taxi_2024_df.loc[mask_2, ['RatecodeID']] = 2
    ny_taxi_2024_df.loc[mask_3, ['RatecodeID']] = 4
    return ny_taxi_2024_df
    

In [165]:
impute_ratecodeIDs(ny_taxi_2024_df)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,1.0,N,16,7,0,-4.13,0.0,0.5,0.00,0.0,1.0,-2.63,NaN,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,1.0,N,252,16,0,-1.68,0.0,0.5,0.00,0.0,1.0,-0.18,NaN,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,1.0,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,NaN,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,1.0,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,NaN,0.0


In [166]:
ny_taxi_2024_df['RatecodeID'].value_counts()

RatecodeID
1.0    38261195
2.0     2376302
5.0      203414
4.0      196787
3.0      131965
6.0          57
Name: count, dtype: int64

In [167]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09937478321445956
0.0


In [168]:
ny_taxi_2024_df['congestion_surcharge'].value_counts()

congestion_surcharge
 2.50    33601544
 0.00     2981167
-2.50      495742
 1.00          18
 0.75          12
-0.75           3
 2.00           1
 2.52           1
Name: count, dtype: int64

In [169]:
ny_taxi_2024_df[ny_taxi_2024_df['congestion_surcharge'] == -0.75]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2796848,2,2024-01-31 18:27:00,2024-01-31 18:28:04,1.0,0.00,6.0,N,79,79,4,-2.5,0.0,-0.5,0.0,0.0,-1.0,-4.75,-0.75,0.0
1919717,2,2024-03-19 16:46:50,2024-03-19 16:47:23,1.0,0.02,6.0,N,170,170,2,0.0,0.0,-0.5,0.0,0.0,-1.0,-2.25,-0.75,0.0
2062349,2,2024-10-19 19:53:54,2024-10-19 19:55:55,1.0,0.10,6.0,N,164,164,3,-5.5,0.0,-0.5,0.0,0.0,-1.0,-7.75,-0.75,0.0


In [170]:
def impute_negatives(ny_taxi_2024_df):
    negatives = ['fare_amount', 'extra', 'tip_amount', 'tolls_amount', 'mta_tax', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']
    mask = (ny_taxi_2024_df[negatives] < 0).any(axis=1)
    mask_2 = (np.isnan(ny_taxi_2024_df['congestion_surcharge']))
    ny_taxi_2024_df.loc[mask, negatives] = ny_taxi_2024_df.loc[mask, negatives].abs()
    ny_taxi_2024_df.loc[mask_2, ['congestion_surcharge']] = 0
    return ny_taxi_2024_df

In [171]:
impute_negatives(ny_taxi_2024_df)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,1.0,N,16,7,0,4.13,0.0,0.5,0.00,0.0,1.0,2.63,0.0,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,1.0,N,252,16,0,1.68,0.0,0.5,0.00,0.0,1.0,0.18,0.0,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,1.0,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,0.0,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,1.0,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,0.0,0.0


In [172]:
for column in ny_taxi_2024_df:

    print(ny_taxi_2024_df[column].isna().sum()/len(ny_taxi_2024_df[column]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [173]:
for column in ny_taxi_2024_df:
    print(ny_taxi_2024_df[column].value_counts())

VendorID
2    31451503
1     9715918
6        2069
7         230
Name: count, dtype: int64
tpep_pickup_datetime
2024-06-14 19:33:00    17
2024-04-02 18:15:00    17
2024-11-26 11:00:00    16
2024-11-15 10:30:00    16
2024-11-15 14:00:00    16
                       ..
2024-07-31 07:29:16     1
2024-07-31 07:54:32     1
2024-07-31 07:38:24     1
2024-02-24 19:38:32     1
2024-12-31 23:02:30     1
Name: count, Length: 20049544, dtype: int64
tpep_dropoff_datetime
2024-04-20 00:00:00    25
2024-12-13 07:59:20    24
2024-12-13 08:02:00    23
2024-12-13 07:59:19    23
2024-06-13 00:00:00    22
                       ..
2024-04-17 10:08:45     1
2024-08-31 22:10:35     1
2024-04-17 10:39:06     1
2024-04-17 10:28:10     1
2024-12-31 23:23:16     1
Name: count, Length: 20030251, dtype: int64
passenger_count
1.0    33125289
2.0     5410774
3.0     1282073
4.0      814889
5.0      320613
6.0      215798
8.0         192
7.0          56
9.0          36
Name: count, dtype: int64
trip_distance
0.00  

In [174]:
ny_taxi_2024_df[ny_taxi_2024_df['trip_distance'] == 223463.68]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3665784,2,2024-12-31 22:18:00,2024-12-31 23:05:00,1.0,223463.68,1.0,N,61,164,0,44.85,0.0,0.5,2.12,0.0,1.0,50.97,0.0,0.0


In [175]:
def trip_distance_weird_maxes(ny_taxi_2024_df):
    long = (ny_taxi_2024_df['trip_distance'] > 50)
    cost = (ny_taxi_2024_df['total_amount'] < 100)
    params = long & cost
    ny_taxi_2024_df.loc[params, ['trip_distance']] = (ny_taxi_2024_df.loc[params, ['fare_amount']] / 5.2)
    ny_taxi_2024_df.loc[params, ['trip_distance']] = (ny_taxi_2024_df.loc[params, ['trip_distance']] * 1.05)
    return ny_taxi_2024_df

In [176]:
trip_distance_weird_maxes(ny_taxi_2024_df)

ny_taxi_2024_df['trip_distance'].value_counts()

trip_distance
0.00      776305
0.90      519150
1.00      517069
0.80      508274
1.10      502953
           ...  
64.92          1
70.21          1
68.75          1
54.95          1
197.97         1
Name: count, Length: 7652, dtype: int64

In [177]:
ny_taxi_2024_df[ny_taxi_2024_df['trip_distance'] == 197.97]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3628231,2,2024-12-24 20:16:18,2024-12-24 23:52:37,1.0,197.97,4.0,N,230,265,0,442.91,0.0,0.0,0.0,0.0,1.0,443.91,0.0,0.0


In [178]:
ny_taxi_2024_df[ny_taxi_2024_df['trip_distance'] == 67.32]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
266673,2,2024-01-04 15:23:03,2024-01-04 17:20:56,4.0,67.32,4.0,N,132,265,1,350.0,0.0,0.0,10.0,6.94,1.0,369.69,0.0,1.75


In [179]:
ny_taxi_2024_df[ny_taxi_2024_df['trip_distance'] > 50]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
8274,2,2024-01-01 01:18:30,2024-01-01 02:48:00,1.0,54.94,4.0,N,132,265,2,243.10,1.0,0.0,0.00,13.88,1.0,260.73,0.0,1.75
16093,2,2024-01-01 02:37:39,2024-01-01 04:25:31,1.0,54.81,4.0,N,158,265,2,299.10,1.0,0.5,0.00,13.88,1.0,317.98,2.5,0.00
25906,2,2024-01-01 08:09:04,2024-01-01 09:18:26,1.0,60.93,4.0,N,132,265,1,150.00,0.0,0.0,33.50,14.75,1.0,201.00,0.0,1.75
29432,2,2024-01-01 10:42:19,2024-01-01 11:35:08,1.0,51.83,4.0,N,230,265,1,357.90,0.0,0.0,0.00,12.75,1.0,371.65,0.0,0.00
36881,2,2024-01-01 13:30:49,2024-01-01 14:27:09,1.0,51.93,4.0,N,132,265,1,185.70,0.0,0.5,37.44,0.00,1.0,226.39,0.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3610560,2,2024-12-22 19:13:34,2024-12-22 22:15:21,1.0,164.40,4.0,N,68,265,0,455.09,0.0,0.0,0.00,28.76,1.0,484.85,0.0,0.00
3620166,2,2024-12-23 21:01:23,2024-12-23 22:29:22,1.0,61.12,4.0,N,42,265,0,226.19,0.0,0.0,36.00,13.38,1.0,276.57,0.0,0.00
3628231,2,2024-12-24 20:16:18,2024-12-24 23:52:37,1.0,197.97,4.0,N,230,265,0,442.91,0.0,0.0,0.00,0.00,1.0,443.91,0.0,0.00
3639043,2,2024-12-26 18:29:54,2024-12-26 20:15:34,1.0,68.88,4.0,N,186,265,0,202.50,0.0,0.0,0.00,3.18,1.0,206.68,0.0,0.00


In [180]:
ny_taxi_2024_df[ny_taxi_2024_df['fare_amount'] < 50]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,1.0,N,16,7,0,4.13,0.0,0.5,0.00,0.0,1.0,2.63,0.0,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,1.0,N,252,16,0,1.68,0.0,0.5,0.00,0.0,1.0,0.18,0.0,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,1.0,N,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,0.0,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,1.0,N,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,0.0,0.0


In [181]:
# do we even want the data of trips coming from or going out of NY? i.e zone 265?

def drop_unknowns(ny_taxi_2024_df):
    drop_rows = (ny_taxi_2024_df['PULocationID'] == 265) | (ny_taxi_2024_df['DOLocationID'] == 265) | (
        ny_taxi_2024_df['PULocationID'] == 264) | (ny_taxi_2024_df['DOLocationID'] == 264
    ) | (ny_taxi_2024_df['payment_type'] == 3) | (ny_taxi_2024_df['payment_type'] == 5) | (
        ny_taxi_2024_df['payment_type'] == 6 | (ny_taxi_2024_df['fare_amount'] > ny_taxi_2024_df['total_amount'])
    )
    return ny_taxi_2024_df.drop(ny_taxi_2024_df[drop_rows].index, inplace=True)


In [182]:
drop_unknowns(ny_taxi_2024_df)

In [183]:
ny_taxi_2024_df['trip_distance'].value_counts()

trip_distance
0.00     100475
1.40      14566
1.30      14422
1.20      14344
1.50      14254
          ...  
23.74         1
22.98         1
23.59         1
24.42         1
24.76         1
Name: count, Length: 2892, dtype: int64

In [184]:
ny_taxi_2024_df[ny_taxi_2024_df['trip_distance'] == 206.30]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee


In [185]:
len(ny_taxi_2024_df)

1732681

In [186]:
def impute_store_and_fwd_flag(ny_taxi_2024_df):
    no_mask = (ny_taxi_2024_df['store_and_fwd_flag'] == 'N')
    yes_mask = (ny_taxi_2024_df['store_and_fwd_flag'] == 'Y')

    ny_taxi_2024_df.loc[no_mask, ['store_and_fwd_flag']] = 0
    ny_taxi_2024_df.loc[yes_mask, ['store_and_fwd_flag']] = 1

    return ny_taxi_2024_df

In [187]:
impute_store_and_fwd_flag(ny_taxi_2024_df)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2901612,1,2024-01-18 16:16:29,2024-01-18 16:28:21,1.0,0.90,1.0,0,186,100,0,9.30,2.5,0.5,3.16,0.0,1.0,18.96,0.0,0.0
2901612,1,2024-02-16 10:18:17,2024-02-16 10:31:03,1.0,1.20,1.0,0,100,163,0,11.40,0.0,0.5,3.08,0.0,1.0,18.48,0.0,0.0
2901612,2,2024-03-29 11:30:47,2024-03-29 11:44:05,1.0,1.63,1.0,0,141,163,2,13.50,0.0,0.5,0.00,0.0,1.0,17.50,2.5,0.0
3149557,1,2024-03-31 21:46:36,2024-03-31 21:55:52,2.0,1.70,1.0,0,262,239,2,11.40,3.5,0.5,0.00,0.0,1.0,16.40,2.5,0.0
3152056,1,2024-03-31 22:37:34,2024-03-31 22:54:54,1.0,4.20,1.0,0,237,148,2,19.80,3.5,0.5,0.00,0.0,1.0,24.80,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,1.0,10.71,1.0,0,16,7,0,4.13,0.0,0.5,0.00,0.0,1.0,2.63,0.0,0.0
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,1.0,4.56,1.0,0,252,16,0,1.68,0.0,0.5,0.00,0.0,1.0,0.18,0.0,0.0
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,1.0,3.94,1.0,0,181,255,0,4.46,0.0,0.5,5.19,0.0,1.0,11.15,0.0,0.0
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,1.0,4.20,1.0,0,165,61,0,27.07,0.0,0.5,0.00,0.0,1.0,28.57,0.0,0.0


In [188]:
ny_taxi_2024_df['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
0    1732604
1         77
Name: count, dtype: int64

In [189]:
ny_taxi_2024_df[ny_taxi_2024_df['passenger_count'] >= 4]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
3185704,2,2024-05-30 19:44:36,2024-05-30 19:58:27,4.0,0.84,1.0,0,230,230,2,12.8,2.50,0.5,0.0,0.0,1.0,19.30,2.5,0.00
3220010,2,2024-05-31 06:53:18,2024-05-31 06:58:53,5.0,1.45,1.0,0,13,158,2,9.3,0.00,0.5,0.0,0.0,1.0,13.30,2.5,0.00
3241623,2,2024-05-31 11:09:24,2024-05-31 11:33:12,4.0,2.11,1.0,0,249,161,2,20.5,0.00,0.5,0.0,0.0,1.0,24.50,2.5,0.00
3251379,2,2024-05-31 13:29:50,2024-05-31 13:39:18,6.0,1.19,1.0,0,43,74,2,10.0,0.00,0.5,0.0,0.0,1.0,11.50,0.0,0.00
3280677,2,2024-05-31 17:04:16,2024-05-31 17:11:44,5.0,0.68,1.0,0,162,161,2,7.9,2.50,0.5,0.0,0.0,1.0,14.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339709,1,2024-12-31 23:56:10,2025-01-01 00:21:35,4.0,11.10,1.0,1,138,181,2,45.7,7.75,0.5,0.0,0.0,1.0,54.95,0.0,1.75
3339764,1,2024-12-31 23:18:32,2024-12-31 23:43:20,4.0,7.30,1.0,0,74,144,2,33.1,3.50,0.5,0.0,0.0,1.0,38.10,2.5,0.00
3340058,2,2024-12-31 23:21:31,2024-12-31 23:30:27,4.0,1.20,1.0,0,246,158,2,10.0,1.00,0.5,0.0,0.0,1.0,15.00,2.5,0.00
3340213,1,2024-12-31 23:00:07,2024-12-31 23:11:43,4.0,0.90,1.0,0,141,161,2,10.0,3.50,0.5,0.0,0.0,1.0,15.00,2.5,0.00


In [190]:
ny_taxi_2024_df[ny_taxi_2024_df['payment_type'] == 4]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2901612,2,2024-05-28 08:33:03,2024-05-28 08:38:50,1.0,0.71,1.0,0,186,170,4,7.2,0.0,0.5,0.0,0.0,1.0,11.20,2.5,0.00
3169784,2,2024-05-30 17:20:14,2024-05-30 17:38:12,3.0,3.41,1.0,0,158,163,4,19.1,2.5,0.5,0.0,0.0,1.0,25.60,2.5,0.00
3186643,2,2024-05-30 19:10:48,2024-05-30 19:11:25,1.0,0.03,1.0,0,234,234,4,3.0,2.5,0.5,0.0,0.0,1.0,9.50,2.5,0.00
3212428,2,2024-05-30 23:31:35,2024-05-30 23:37:37,1.0,0.92,1.0,0,249,231,4,7.9,1.0,0.5,0.0,0.0,1.0,12.90,2.5,0.00
3216321,2,2024-05-31 01:02:34,2024-05-31 01:04:15,2.0,0.25,1.0,0,164,234,4,4.4,1.0,0.5,0.0,0.0,1.0,9.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341957,2,2024-12-31 23:42:29,2024-12-31 23:48:47,1.0,0.68,1.0,0,161,164,4,7.2,1.0,0.5,0.0,0.0,1.0,12.20,2.5,0.00
3341958,2,2024-12-31 23:42:29,2024-12-31 23:48:47,1.0,0.68,1.0,0,161,164,4,7.2,1.0,0.5,0.0,0.0,1.0,12.20,2.5,0.00
3341960,2,2024-12-31 23:51:07,2025-01-01 00:01:02,3.0,1.69,1.0,0,164,68,4,11.4,1.0,0.5,0.0,0.0,1.0,16.40,2.5,0.00
3342043,2,2024-12-31 23:01:43,2024-12-31 23:05:57,1.0,0.49,1.0,0,263,236,4,5.8,1.0,0.5,0.0,0.0,1.0,10.80,2.5,0.00


In [191]:
ny_taxi_2024_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [192]:
ny_taxi_2024_df = ny_taxi_2024_df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance', 'RatecodeID', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee', 'tip_amount', 'total_amount']]

In [194]:
display(ny_taxi_2024_df.head(10))
display(ny_taxi_2024_df.tail(10))

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,trip_distance,RatecodeID,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,tip_amount,total_amount
2901612,1,2024-01-18 16:16:29,2024-01-18 16:28:21,186,100,0.90,1.0,9.30,2.5,0.5,0.0,1.0,0.0,0.0,3.16,18.96
2901612,1,2024-02-16 10:18:17,2024-02-16 10:31:03,100,163,1.20,1.0,11.40,0.0,0.5,0.0,1.0,0.0,0.0,3.08,18.48
2901612,2,2024-03-29 11:30:47,2024-03-29 11:44:05,141,163,1.63,1.0,13.50,0.0,0.5,0.0,1.0,2.5,0.0,0.00,17.50
3149557,1,2024-03-31 21:46:36,2024-03-31 21:55:52,262,239,1.70,1.0,11.40,3.5,0.5,0.0,1.0,2.5,0.0,0.00,16.40
3152056,1,2024-03-31 22:37:34,2024-03-31 22:54:54,237,148,4.20,1.0,19.80,3.5,0.5,0.0,1.0,2.5,0.0,0.00,24.80
3156955,2,2024-03-01 02:56:06,2024-03-01 03:07:52,79,13,2.61,1.0,17.84,0.0,0.5,0.0,1.0,0.0,0.0,4.37,26.21
3157690,1,2024-03-01 06:43:55,2024-03-01 06:53:11,158,48,0.00,1.0,13.01,0.0,0.5,0.0,1.0,0.0,0.0,0.00,17.01
3158122,1,2024-03-01 07:33:05,2024-03-01 07:45:52,238,262,0.00,1.0,20.54,0.0,0.5,0.0,1.0,0.0,0.0,0.00,24.54
3159420,2,2024-03-01 10:32:20,2024-03-01 10:37:44,164,186,0.47,1.0,11.74,0.0,0.5,0.0,1.0,0.0,0.0,3.15,18.89
3160931,1,2024-03-01 16:09:54,2024-03-01 16:20:41,236,140,0.00,1.0,13.19,0.0,0.5,0.0,1.0,0.0,0.0,0.00,17.19


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,trip_distance,RatecodeID,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,tip_amount,total_amount
3668361,1,2024-12-31 23:21:36,2024-12-31 23:32:10,188,61,1.60,1.0,13.28,0.0,0.5,0.00,1.0,0.0,0.0,0.00,14.78
3668362,2,2024-12-31 23:02:30,2024-12-31 23:23:16,37,37,2.47,1.0,0.90,0.0,0.5,0.00,1.0,0.0,0.0,0.00,0.60
3668363,2,2024-12-31 23:04:03,2024-12-31 23:14:58,137,79,1.96,1.0,7.72,0.0,0.5,0.00,1.0,0.0,0.0,0.00,3.72
3668364,1,2024-12-31 23:14:53,2024-12-31 23:35:13,145,164,2.90,1.0,21.94,0.0,0.5,6.94,1.0,0.0,0.0,0.00,32.88
3668365,2,2024-12-31 23:04:32,2024-12-31 23:19:51,37,36,2.49,1.0,0.38,0.0,0.5,0.00,1.0,0.0,0.0,0.00,6.12
3668366,2,2024-12-31 23:32:00,2024-12-31 23:56:00,16,7,10.71,1.0,4.13,0.0,0.5,0.00,1.0,0.0,0.0,0.00,2.63
3668367,2,2024-12-31 23:05:00,2024-12-31 23:18:00,252,16,4.56,1.0,1.68,0.0,0.5,0.00,1.0,0.0,0.0,0.00,0.18
3668368,2,2024-12-31 23:03:16,2024-12-31 23:28:35,181,255,3.94,1.0,4.46,0.0,0.5,0.00,1.0,0.0,0.0,5.19,11.15
3668369,1,2024-12-31 23:15:33,2024-12-31 23:36:29,165,61,4.20,1.0,27.07,0.0,0.5,0.00,1.0,0.0,0.0,0.00,28.57
3668370,2,2024-12-31 23:04:36,2024-12-31 23:33:34,225,114,5.76,1.0,2.17,0.0,0.5,0.00,1.0,0.0,0.0,0.00,6.17
